In [1]:
import pyspark
from pyspark.sql.functions import *
sc = pyspark.SparkContext('local[*]')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2024-12-07T20:31:06,114 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import *

In [3]:
# import findspark
# findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
# 1
cleaned_olist_customers_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_customers_dataset.csv", header=True, inferSchema=True)
# 2
cleaned_olist_geolocation_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_geolocation_dataset.csv", header=True, inferSchema=True)
# 3
cleaned_olist_order_items_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_order_items_dataset.csv", header=True, inferSchema=True)
# 4
cleaned_olist_order_payments_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_order_payments_dataset.csv", header=True, inferSchema=True)
# 5
cleaned_olist_order_reviews_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_order_reviews_dataset.csv", header=True, inferSchema=True)
# 6
cleaned_olist_orders_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_orders_dataset.csv", header=True, inferSchema=True)
# 7
cleaned_olist_products_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_products_dataset.csv", header=True, inferSchema=True)
# 8
cleaned_olist_sellers_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_olist_sellers_dataset.csv", header=True, inferSchema=True)
# 9
cleaned_product_category_name_translation_dataset = spark.read.csv("/obligatorio/data/cleaned/cleaned_product_category_name_translation.csv", header=True, inferSchema=True)

Database Schema

In [6]:
# Products

In [7]:
cleaned_olist_products_dataset.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)



In [8]:
cleaned_product_category_name_translation_dataset.printSchema()

root
 |-- product_category_name: string (nullable = true)
 |-- product_category_name_english: string (nullable = true)



In [9]:
dim_products = cleaned_olist_products_dataset.join(
    cleaned_product_category_name_translation_dataset,
    "product_category_name",
    "left"
).select(
    col("product_id"),
    col("product_category_name"),
    col("product_category_name_english"),
    col("product_weight_g").alias("weight_g"),
    col("product_length_cm").alias("length_cm"),
    col("product_height_cm").alias("height_cm"),
    col("product_width_cm").alias("width_cm")
)

In [10]:
dim_products.show(5)

+--------------------+---------------------+-----------------------------+--------+---------+---------+--------+
|          product_id|product_category_name|product_category_name_english|weight_g|length_cm|height_cm|width_cm|
+--------------------+---------------------+-----------------------------+--------+---------+---------+--------+
|7fdbc1ecf5fdf5c98...|        esporte_lazer|               sports_leisure|     213|       16|       10|      13|
|42fe4aefc7e8a45dd...|           automotivo|                         auto|    1300|       20|       20|      20|
|0d79e2972015716ed...|         market_place|                 market_place|    2150|       40|       25|      30|
|a2f4e28e50f60566e...| instrumentos_musi...|          musical_instruments|    5850|       40|        9|     105|
|b2cadfc2d6da74fba...| utilidades_domest...|                   housewares|     400|       16|        9|      12|
+--------------------+---------------------+-----------------------------+--------+---------+---

In [11]:
cleaned_olist_products_dataset.show(5)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|7fdbc1ecf5fdf5c98...|        esporte_lazer|                 57|                       649|                 1|             213|               16|               10|              13|
|42fe4aefc7e8a45dd...|           automotivo|                 59|                       162|                 1|            1300|               20|               20|              20|
|0d79e2972015716ed...|         market_place|                 50|                       257|    

In [12]:
cleaned_product_category_name_translation_dataset.show(5)

+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|           automotivo|                         auto|
|                  pcs|                    computers|
|      eletroportateis|             small_appliances|
|     moveis_decoracao|              furniture_decor|
| fashion_roupa_mas...|         fashion_male_clot...|
+---------------------+-----------------------------+
only showing top 5 rows



In [13]:
# Reviews

In [14]:
cleaned_olist_order_reviews_dataset.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)



In [15]:
dim_reviews = cleaned_olist_order_reviews_dataset.select("*")

In [16]:
dim_reviews.show(5)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|8f9aa50ee40780ba3...|729c445f5f350f88d...|           5|           Muito bom|  Creme maravilhoso...| 2018-05-10 00:00:00|    2018-05-10 16:21:17|
|38bcaf95057f842c3...|ca36cc19c0a413c8f...|           3|                null|  Só recebi um prod...| 2018-04-28 00:00:00|    2018-04-29 01:11:49|
|08ee947fb131a0a99...|0666806082c797ed8...|           5|                null|  Produto lindo, ótimo| 2017-07-08 00:00:00|    2017-07-09 09:59:54|
|1f0c5d31b49a18cc2...|f810273bc84ceae52...|           4|                null|  Entrega uma dia a...| 2018-03-11 00:00:00|   

In [17]:
cleaned_olist_order_reviews_dataset.show(5)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|8f9aa50ee40780ba3...|729c445f5f350f88d...|           5|           Muito bom|  Creme maravilhoso...| 2018-05-10 00:00:00|    2018-05-10 16:21:17|
|38bcaf95057f842c3...|ca36cc19c0a413c8f...|           3|                null|  Só recebi um prod...| 2018-04-28 00:00:00|    2018-04-29 01:11:49|
|08ee947fb131a0a99...|0666806082c797ed8...|           5|                null|  Produto lindo, ótimo| 2017-07-08 00:00:00|    2017-07-09 09:59:54|
|1f0c5d31b49a18cc2...|f810273bc84ceae52...|           4|                null|  Entrega uma dia a...| 2018-03-11 00:00:00|   

In [18]:
# Payments

In [19]:
cleaned_olist_order_payments_dataset.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)



In [20]:
dim_payments = cleaned_olist_order_payments_dataset.select(
    col("order_id"),
    col("payment_type"),
    col("payment_installments"),
    col("payment_value")
).withColumn("payment_id", monotonically_increasing_id())

In [21]:
# Ordeno para que "payment_id" quede primera
dim_payments = dim_payments.select(
    col("payment_id"),
    col("order_id"),
    col("payment_type"),
    col("payment_installments"),
    col("payment_value")
)

In [22]:
dim_payments.show(5)

+----------+--------------------+------------+--------------------+-------------+
|payment_id|            order_id|payment_type|payment_installments|payment_value|
+----------+--------------------+------------+--------------------+-------------+
|         0|9389b64e7d922e3b0...| credit_card|                   4|       125.43|
|         1|91e725244b1ed7224...| credit_card|                   3|       113.33|
|         2|ad9447ca1bb7d23ed...|      boleto|                   1|       224.45|
|         3|41b03a6bad9f1feda...| credit_card|                   1|       220.66|
|         4|9378f7fb4dad9527f...| credit_card|                   1|       117.01|
+----------+--------------------+------------+--------------------+-------------+
only showing top 5 rows



In [23]:
cleaned_olist_order_payments_dataset.show(5)

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|9389b64e7d922e3b0...|                 1| credit_card|                   4|       125.43|
|91e725244b1ed7224...|                 1| credit_card|                   3|       113.33|
|ad9447ca1bb7d23ed...|                 1|      boleto|                   1|       224.45|
|41b03a6bad9f1feda...|                 1| credit_card|                   1|       220.66|
|9378f7fb4dad9527f...|                 1| credit_card|                   1|       117.01|
+--------------------+------------------+------------+--------------------+-------------+
only showing top 5 rows



In [24]:
# Geolocalización

In [25]:
cleaned_olist_geolocation_dataset.printSchema()

root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



In [26]:
dim_geolocation = cleaned_olist_geolocation_dataset.select("*")

In [27]:
dim_geolocation.show(5)

+---------------------------+-------------------+------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|   geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+------------------+----------------+-----------------+
|                       1001|-23.551426655288804|-46.63407394670785|       sao paulo|               SP|
|                       1010|-23.545361513938836| -46.6346895341229|       sao paulo|               SP|
|                       1048|-23.546871901465792|-46.63943652582367|       sao paulo|               SP|
|                       1019|-23.552151883432384|-46.63080224715792|       são paulo|               SP|
|                       1019|-23.551489508194305|-46.63140317736157|       sao paulo|               SP|
+---------------------------+-------------------+------------------+----------------+-----------------+
only showing top 5 rows



In [28]:
cleaned_olist_geolocation_dataset.show(5)

+---------------------------+-------------------+------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|   geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+------------------+----------------+-----------------+
|                       1001|-23.551426655288804|-46.63407394670785|       sao paulo|               SP|
|                       1010|-23.545361513938836| -46.6346895341229|       sao paulo|               SP|
|                       1048|-23.546871901465792|-46.63943652582367|       sao paulo|               SP|
|                       1019|-23.552151883432384|-46.63080224715792|       são paulo|               SP|
|                       1019|-23.551489508194305|-46.63140317736157|       sao paulo|               SP|
+---------------------------+-------------------+------------------+----------------+-----------------+
only showing top 5 rows



In [29]:
# Seller

In [30]:
cleaned_olist_sellers_dataset.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)



In [31]:
dim_sellers = cleaned_olist_sellers_dataset.join(
    dim_geolocation,
    cleaned_olist_sellers_dataset.seller_zip_code_prefix == dim_geolocation.geolocation_zip_code_prefix,
    "left"
).select(
    col("seller_id"),
    col("seller_city"),
    col("seller_state"),
    col("geolocation_lat"),
    col("geolocation_lng")
)

In [32]:
dim_sellers.show(5)

+--------------------+-----------+------------+-------------------+-------------------+
|           seller_id|seller_city|seller_state|    geolocation_lat|    geolocation_lng|
+--------------------+-----------+------------+-------------------+-------------------+
|f6122bc84774df1b3...|    barueri|          SP| -23.53282397861485|-46.890643148704044|
|f6122bc84774df1b3...|    barueri|          SP|-23.538735009677385| -46.88406871524743|
|f6122bc84774df1b3...|    barueri|          SP| -23.53617187717585|-46.885508757720494|
|f6122bc84774df1b3...|    barueri|          SP|-23.537594046588353| -46.89183020500965|
|f6122bc84774df1b3...|    barueri|          SP|-23.537057132120676| -46.89116185892949|
+--------------------+-----------+------------+-------------------+-------------------+
only showing top 5 rows



In [33]:
cleaned_olist_sellers_dataset.show(5)

+--------------------+----------------------+-------------+------------+
|           seller_id|seller_zip_code_prefix|  seller_city|seller_state|
+--------------------+----------------------+-------------+------------+
|f6122bc84774df1b3...|                  6436|      barueri|          SP|
|8132b9bd16876e1b0...|                 88058|florianopolis|          SC|
|a254c682cc01e119f...|                 16500|   cafelandia|          SP|
|cb9fb4ca75d7ba843...|                  3551|    sao paulo|          SP|
|c5ebe6598748b0aea...|                 95910|      lajeado|          RS|
+--------------------+----------------------+-------------+------------+
only showing top 5 rows



In [34]:
# Customer

In [35]:
cleaned_olist_customers_dataset.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [36]:
dim_customers = cleaned_olist_customers_dataset.join(
    dim_geolocation,
    cleaned_olist_customers_dataset.customer_zip_code_prefix == dim_geolocation.geolocation_zip_code_prefix,
    "left"
).select(
    col("customer_id"),
    col("customer_unique_id"),
    col("customer_city"),
    col("customer_state"),
    col("geolocation_lat"),
    col("geolocation_lng")
)

In [37]:
dim_customers.show(5)

+--------------------+--------------------+-------------+--------------+-------------------+-------------------+
|         customer_id|  customer_unique_id|customer_city|customer_state|    geolocation_lat|    geolocation_lng|
+--------------------+--------------------+-------------+--------------+-------------------+-------------------+
|369708cabd9831ea6...|94b731a41867b47c3...|    sao paulo|            SP|-23.512254251563892| -46.50257041614536|
|369708cabd9831ea6...|94b731a41867b47c3...|    sao paulo|            SP|-23.523337980058184| -46.53472451981566|
|369708cabd9831ea6...|94b731a41867b47c3...|    sao paulo|            SP|-23.522752889668638|-46.524084427199654|
|369708cabd9831ea6...|94b731a41867b47c3...|    sao paulo|            SP|-23.521909169859764| -46.52710027116393|
|369708cabd9831ea6...|94b731a41867b47c3...|    sao paulo|            SP|-23.523158336112846| -46.53311089869581|
+--------------------+--------------------+-------------+--------------+-------------------+----

In [38]:
cleaned_olist_customers_dataset.show(5)

+--------------------+--------------------+------------------------+-------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+--------------------+--------------------+------------------------+-------------+--------------+
|369708cabd9831ea6...|94b731a41867b47c3...|                    3636|    sao paulo|            SP|
|cfffbb322333151f8...|0dc0731aafd565941...|                    5093|    sao paulo|            SP|
|39592999581de2960...|3d7fb21a86fa49b9d...|                   35430|   ponte nova|            MG|
|473e95fa97e4514de...|ee301c76bd2b939da...|                   24315|      niteroi|            RJ|
|6419daa620ebb5a4d...|b9137cac4c1f714b3...|                   29285|        piuma|            ES|
+--------------------+--------------------+------------------------+-------------+--------------+
only showing top 5 rows



In [39]:
# Item

In [40]:
cleaned_olist_order_items_dataset.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [41]:
dim_items = cleaned_olist_order_items_dataset \
    .join(dim_products, "product_id", "left") \
    .join(dim_sellers, "seller_id", "left") \
.select(
    col("order_id"),
    col("order_item_id"),
    col("product_id"),
    col("seller_id"),
    col("shipping_limit_date"),
    col("price"),
    col("freight_value")
)

In [42]:
dim_items.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|019636907d07b2433...|            1|2b0ee2d07306f7c9a...|1900267e848ceeba8...|2017-11-22 16:08:57|59.99|        12.76|
|019636907d07b2433...|            1|2b0ee2d07306f7c9a...|1900267e848ceeba8...|2017-11-22 16:08:57|59.99|        12.76|
|019636907d07b2433...|            1|2b0ee2d07306f7c9a...|1900267e848ceeba8...|2017-11-22 16:08:57|59.99|        12.76|
|019636907d07b2433...|            1|2b0ee2d07306f7c9a...|1900267e848ceeba8...|2017-11-22 16:08:57|59.99|        12.76|
|019636907d07b2433...|            1|2b0ee2d07306f7c9a...|1900267e848ceeba8...|2017-11-22 16:08:57|59.99|        12.76|
+--------------------+-------------+------------

In [43]:
cleaned_olist_order_items_dataset.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|0184d4ddb259e1a4c...|            1|aa8627a375771ab01...|e333046ce6517bd8b...|2017-09-07 22:31:42| 75.0|        16.16|
|019636907d07b2433...|            1|2b0ee2d07306f7c9a...|1900267e848ceeba8...|2017-11-22 16:08:57|59.99|        12.76|
|01ad4f241a16bff72...|            1|448f0791c4d2fc5d2...|a888faf2d1baececa...|2017-02-18 18:47:17|309.9|        17.38|
|01b6d15407d425529...|            1|78c3942245093508f...|4830e40640734fc1c...|2018-03-19 15:35:37|29.99|        12.79|
|02f4865eba77caf30...|            1|a50acd33ba7a8da8e...|8581055ce74af1dab...|2017-04-26 03:15:05|117.3|        22.29|
+--------------------+-------------+------------

In [44]:
# Order

In [45]:
cleaned_olist_orders_dataset.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [ ]:
# Unión de las tablas necesarias
fact_order = cleaned_olist_orders_dataset.join(dim_items, "order_id") \
    .join(dim_reviews, "order_id", "left") \
    .join(dim_payments, "order_id", "left") \
    .join(dim_customers, "customer_id", "left") \
    .groupBy(
        "order_id",
        "product_id",
        "customer_id",
        "seller_id",
        "review_id",
        "payment_id",
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date"
    ).agg(
        sum("price").alias("total_price"),
        sum("freight_value").alias("total_freight_value"),
        sum("payment_value").alias("total_payment_value"), 
        expr("datediff(order_delivered_customer_date, order_approved_at)").alias("diff_order_delivered_customer_vs_order_approved_at")
    ).withColumn("total_items", lit(dim_items.count()))

In [47]:
fact_order.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- payment_id: long (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- total_price: double (nullable = true)
 |-- total_freight_value: double (nullable = true)
 |-- total_payment_value: double (nullable = true)
 |-- diff_order_delivered_customer_vs_order_approved_at: integer (nullable = true)
 |-- total_items: integer (nullable = false)



In [48]:
fact_order.show(5)

2024-12-07T20:32:53,500 WARN [Executor task launch worker for task 0.0 in stage 102.0 (TID 93)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2024-12-07T20:32:54,400 WARN [Executor task launch worker for task 0.0 in stage 102.0 (TID 93)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2024-12-07T20:32:54,908 WARN [Executor task launch worker for task 0.0 in stage 102.0 (TID 93)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2024-12-07T20:32:55,407 WARN [Executor task launch worker for task 0.0 in stage 102.0 (TID 93)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2024-12-07T20:32:55,940 WARN [Executor task launch worker for task 0

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+-------------------+----------------------------+-----------------------------+------------------+-------------------+--------------------+--------------------------------------------------+-----------+
|            order_id|          product_id|         customer_id|           seller_id|           review_id|payment_id|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|       total_price|total_freight_value| total_payment_value|diff_order_delivered_customer_vs_order_approved_at|total_items|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------------------------+-------------------+----------------------------+-----------------------------+------------------+-------------------+--------------------+-----------------------------

In [49]:
cleaned_olist_orders_dataset.show(5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|6514b8ad8028c9f2c...|9bdf08b4b3b52b552...|   delivered|     2017-05-16 13:10:30|2017-05-16 13:22:11|         2017-05-22 10:07:46|          2017-05-26 12:55:51|          2017-06-07 00:00:00|
|d65c179a817be2297...|e20e5dab42cf2fc3a...|   delivered|     2017-08-09 21:43:53|2017-08-09 22:03:51|         2017-08-10 16:18:52|          2017-08-18 21:09:45|          2017-09-04 00:00:00|
|f09942b1e1419e69f...|b40a130a36dc28cdb...|  

In [50]:
directory_results = "obligatorio/data/processed"

In [51]:
_path = f"hdfs://localhost:9000/{directory_results}/"
_path

'hdfs://localhost:9000/obligatorio/data/processed/'

In [ ]:
spark.conf.set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")

In [52]:
dim_products.write.csv(_path+"/dim_products.csv", header=True, mode="overwrite")

In [53]:
dim_reviews.write.csv(_path+"/dim_reviews.csv", header=True, mode="overwrite")

In [54]:
dim_payments.write.csv(_path+"/dim_payments.csv", header=True, mode="overwrite")

In [56]:
dim_sellers.write.csv(_path+"/dim_sellers.csv", header=True, mode="overwrite")

In [57]:
dim_customers.write.csv(_path+"/dim_customers.csv", header=True, mode="overwrite")

In [58]:
dim_items.write.csv(_path+"/dim_items.csv", header=True, mode="overwrite")

In [59]:
fact_order.write.csv(_path+"/fact_order.csv", header=True, mode="overwrite")

2024-12-07T20:42:58,509 WARN [Executor task launch worker for task 1.0 in stage 162.0 (TID 152)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2024-12-07T20:42:58,561 WARN [Executor task launch worker for task 0.0 in stage 162.0 (TID 151)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


2024-12-07T20:42:58,854 WARN [Executor task launch worker for task 1.0 in stage 162.0 (TID 152)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2024-12-07T20:42:58,927 WARN [Executor task launch worker for task 0.0 in stage 162.0 (TID 151)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2024-12-07T20:42:59,288 WARN [Executor task launch worker for task 1.0 in stage 162.0 (TID 152)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2024-12-07T20:42:59,293 WARN [Executor task launch worker for task 0.0 in stage 162.0 (TID 151)] org.apache.spark.sql.catalyst.expressions.RowBasedKeyValueBatch - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2024-12-07T20:42:59,634 WARN [Executor task launch worker for ta

In [60]:
fact_order.count()

105245